In [2]:
from typing import Callable, Dict, Iterable, Self
from numpy.typing import NDArray
import datasets
import tensorflow as tf
import numpy as np
import einops
import pandas as pd
import pickle

# import flagon

In [7]:
with open("data/solar_home_data.pkl", 'rb') as f:
    data = pickle.load(f)

def get_customer_data(customer=1):
    idx = np.arange(24, len(data[customer]))
    expanded_idx = np.array([np.arange(i - 24, i - 1) for i in idx])
    return data[customer][expanded_idx].reshape(len(data[customer]) - 24, -1), data[customer][idx, 0]

In [10]:
get_customer_data(1)[0].shape

(8736, 92)

In [91]:
def create_model() -> tf.keras.Model:
    inputs = tf.keras.Input((92))
    # x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Reshape((23, 4))(inputs)
    # x = tf.keras.layers.Conv1D(32, 5, activation="relu")(x)
    # x = tf.keras.layers.Conv1D(32, 3, activation="relu")(x)
    # x = tf.keras.layers.Conv1D(32, 2, activation="relu")(x)
    x = tf.keras.layers.LSTM(3)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(50, activation="relu")(x)
    x = tf.keras.layers.Dense(1, activation="linear")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"]
    )
    return model

In [13]:
X, Y = get_customer_data(1)

In [92]:
model = create_model()
model.fit(X, Y, epochs=10)

Epoch 1/10
273/273 [==============================] - 2s 2ms/step - loss: 1.2970 - mean_absolute_error: 0.8739
Epoch 2/10
273/273 [==============================] - 1s 2ms/step - loss: 1.0270 - mean_absolute_error: 0.7899
Epoch 3/10
273/273 [==============================] - 1s 2ms/step - loss: 1.0146 - mean_absolute_error: 0.7844
Epoch 4/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9957 - mean_absolute_error: 0.7750
Epoch 5/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9817 - mean_absolute_error: 0.7654
Epoch 6/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9599 - mean_absolute_error: 0.7559
Epoch 7/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9560 - mean_absolute_error: 0.7541
Epoch 8/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9553 - mean_absolute_error: 0.7512
Epoch 9/10
273/273 [==============================] - 1s 2ms/step - loss: 0.9416 - mean_absolute_error: 0.7476
E

In [93]:
model.predict(X[Y < 0])

86/86 [==============================] - 0s 1ms/step


array([[ 1.4047612 ],
       [ 0.8758196 ],
       [-0.9858786 ],
       ...,
       [ 0.24838148],
       [ 0.38292974],
       [ 1.2833661 ]], dtype=float32)

In [94]:
Y[Y < 0]

array([-0.594, -0.862, -1.591, ..., -0.444, -0.283, -0.364])